In [1]:
import os
import warnings

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler

from rkcompetition.utils.data_preprocessing import *

warnings.filterwarnings("ignore")

In [2]:
inner_actions, outer_actions, type_contract, train, sample, outer_actions_in_train = read_raw_data('../data/raw/')

inner_actions.event_date = pd.to_datetime(inner_actions.event_date)
outer_actions.event_date = pd.to_datetime(outer_actions.event_date)
outer_actions_in_train.event_date = pd.to_datetime(outer_actions_in_train.event_date)

# Long to wide transformation


In [3]:
# transform outer actions
outer_second_df = outer_actions.loc[outer_actions.event_type.isin(outer_actions_in_train.event_type)]
outer_pivot = long_to_wide_with_statistics(outer_actions_in_train, 'outer', cluster_data = False)
outer_pivot
# transform inner actions
inner_pivot = long_to_wide_with_statistics(inner_actions, 'inner', cluster_data = False)
inner_pivot

,inner_Абонент оставил отзыв в Личном кабинете,inner_Активация ГП с IVR,inner_Активация гарантированного платежа,inner_Активация точки подключения на сутки,inner_Без границ бесплатно. Включение,inner_Без границ бесплатно. Отключение,inner_Без границ платно. Включение,inner_Без границ платно. Отключение,inner_Блокировка,inner_Блокировка лицевого счета,...,inner_Удаление телефона из записи,inner_Футболка Ситилинк,inner_Чек лист выполненных работ при регистрации,inner_total_actions,inner_max_actions,inner_min_actions,inner_mean_actions,inner_std_actions,inner_unique_actions,inner_unique_actions_fraction
contract_id,,,,,,,,,,,,,,,,,,,,,
3453,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1,1,0,0.012048,0.109101,1,1.000000
3454,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1,1,0,0.012048,0.109101,1,1.000000
3456,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,4,1,0,0.048193,0.214173,4,1.000000
3457,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,4,4,0,0.048193,0.436404,1,0.250000
3458,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,23,20,0,0.277108,2.202481,2,0.086957
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
274601,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,6,2,0,0.072289,0.301927,5,0.833333
274710,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,7,1,0,0.084337,0.277893,7,1.000000
274782,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,8,4,0,0.096386,0.481325,5,0.625000


# log per day


In [4]:
# calculate week related statistics
inner_actions_week_tab = get_week_statistic(inner_actions, prefix = 'inner_week', get_difference = False)
outer_actions_week_tab = get_week_statistic(outer_actions_in_train, prefix = 'outer', get_difference = False)

keep_from_week_tabs = ['last_action']

inner_actions_week_tab = inner_actions_week_tab.loc[:, [col for col in inner_actions_week_tab.columns if 'last_action' in col][0]]
outer_actions_week_tab = outer_actions_week_tab.loc[:, [col for col in outer_actions_week_tab.columns if 'last_action' in col][0]]

In [5]:
# merging
df_train = train.merge(inner_pivot.reset_index(), on = 'contract_id', how = 'left')
df_train = df_train.merge(type_contract.drop_duplicates(), on = 'contract_id', how = 'left')
df_train = df_train.merge(outer_pivot.reset_index(), on = 'contract_id', how = 'left')




# df_train = df_train.merge(inner_actions_week_tab, left_on = 'contract_id', right_index = True, how = 'left')
# df_train = df_train.merge(outer_actions_week_tab, left_on = 'contract_id', right_index = True, how = 'left')

df_train['outer_more_than_inner'] = df_train.apply(lambda x: True if x.inner_total_actions < x.outer_total_actions else False, axis = 1)

# dropping
df_train = df_train.drop('contract_id', axis = 1)
df_train = df_train.fillna(0)
df_train.columns = df_train.columns.astype(str)
df_train

,blocked,inner_Абонент оставил отзыв в Личном кабинете,inner_Активация ГП с IVR,inner_Активация гарантированного платежа,inner_Активация точки подключения на сутки,inner_Без границ бесплатно. Включение,inner_Без границ бесплатно. Отключение,inner_Без границ платно. Включение,inner_Без границ платно. Отключение,inner_Блокировка,...,outer_www.rt.ru,outer_www.sampo.ru,outer_total_actions,outer_max_actions,outer_min_actions,outer_mean_actions,outer_std_actions,outer_unique_actions,outer_unique_actions_fraction,outer_more_than_inner
0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,83.0,24.0,0.0,0.425641,1.888795,26.0,0.313253,True
1,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,10.0,3.0,0.0,0.051282,0.331950,5.0,0.500000,True
2,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,False
3,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,False
4,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6797,-1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,False
6798,-1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,484.0,249.0,0.0,2.482051,19.399800,9.0,0.018595,False
6799,-1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,9.0,1.0,0.0,0.046154,0.209818,9.0,1.000000,True
6800,-1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,False


In [6]:
# substrings for unification
unify_keys = ['storage', 'broadcast', 'бессмерти', 'iptv', 
              'kino', 'camera', 'rupor', 'speedtest', 'скидка', 'tracker', 'lk','wifi', 'pay', 'stat', 'capsule', 'bify', 'sz', 'fnc'
              ]

# substring for unification without dropping columns
unify_without_drop = ['rt.ru', 'sampo.ru']

# pattern for unification data
patterns = {'strange.sampo.ru': r'[a-z].....*.sampo.ru'}

for key in unify_keys:
    df_train = unify_drop_columns(df_train, col_tag = key)

for name, pattern in patterns.items():
    df_train = unify_drop_columns(df_train, pattern = pattern, col_name = 'strange.sampo.ru')

for key in unify_without_drop:
    df_train = unify_drop_columns(df_train, col_tag = key, drop = False)

In [7]:
corr = 0.03
top_features = df_train.loc[df_train.blocked > -1].corr()['blocked'].dropna().abs() > corr
top_features = top_features.loc[top_features == True].index
df_train = df_train[top_features]
list(top_features)

['blocked',
 'inner_Активация гарантированного платежа',
 'inner_Активация точки подключения на сутки',
 'inner_Без границ бесплатно. Включение',
 'inner_Блокировка',
 'inner_Включение интернета на 20 минут',
 'inner_Включение интернета на 20 минут с IVR',
 'inner_Внутреннее сообщение',
 'inner_Возвращение клиента',
 'inner_Гарантированный платеж за деньги',
 'inner_Добавление в Обращались с номеров',
 'inner_Закрепление сообщения',
 'inner_Изменение данных клиента',
 'inner_Изменение истории',
 'inner_Информер ВК. Показ',
 'inner_Оборудование',
 'inner_Обращение в службу заботы о клиентах',
 'inner_Операция с mac-адресом',
 'inner_Отключение услуги Автоплатёж',
 'inner_Отключение услуги Аренда ТВ-приставки',
 'inner_Перенос денежных средств',
 'inner_Получение подарка в ЛК: 1 месяц бесплатного просмотра IPTV',
 'inner_Получение подарка в ЛК: сутки интернета на максимальной скорости',
 'inner_Предложение IPTV',
 'inner_Принято заявление и доп. соглашение по переезду',
 'inner_Продление

In [8]:
# sample data and train data
df_test = df_train.loc[df_train.blocked == -1]
df_train = df_train.loc[df_train.blocked > -1]

y = df_train.pop('blocked')

x = df_train

In [9]:
C = 30
thresh = 0.52

parameters = {}
lr = LogisticRegression(max_iter=10000, class_weight = 'balanced', C = C# penalty = 'elasticnet', solver= 'saga', l1_ratio = 0.2
                        )

scaler = MinMaxScaler()

pipe1 = Pipeline(steps=[("scaler", scaler), ("logistic", lr)])

clf = GridSearchCV(pipe1, parameters,  scoring = 'f1_macro')
clf.fit(x, y)
print(clf.best_score_)

create_sample(clf, df_test, sample, thresh = thresh, name = 'corr_{}_C_{}_thresh_{}.csv'.format(corr, C, thresh))

0.6005430939411258
240


In [10]:
pd.DataFrame().from_dict([{name: val for name, val in zip(df_train.columns, *clf.best_estimator_['logistic'].coef_)}]).T.abs().sort_values(by = 0, ascending = False)[:40]

,0
inner_Отключение услуги Автоплатёж,11.107965
inner_Отключение услуги Аренда ТВ-приставки,8.596810
бессмерти,7.943983
outer_total_actions,7.136708
outer_mean_actions,7.136708
inner_Смена IP-адреса,6.172266
inner_Принято заявление и доп. соглашение по переезду,4.910032
inner_Операция с mac-адресом,4.411129
inner_Турбокнопка бесплатно,4.398625
storage,3.916390
